In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Path to your unzipped dataset
dataset_path = r"C:\\Users\\MSReddy\\OneDrive\\Documents\\VS code\\Garbage_Class_DL_ML\\TrashType_Image_Dataset"
img_size = 150
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2
)

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)


Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [7]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_gen, validation_data=val_gen, epochs=10)

c:\Users\MSReddy\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\MSReddy\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 57s 849ms/step - accuracy: 0.2632 - loss: 2.1622 - val_accuracy: 0.4652 - val_loss: 1.3449
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 53s 825ms/step - accuracy: 0.4527 - loss: 1.3913 - val_accuracy: 0.5229 - val_loss: 1.2118
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 583ms/step - accuracy: 0.5106 - loss: 1.2506 - val_accuracy: 0.5308 - val_loss: 1.1557
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 463ms/step - accuracy: 0.5584 - loss: 1.1362 - val_accuracy: 0.5865 - val_loss: 1.0793
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 467ms/step - accuracy: 0.6001 - loss: 1.0821 - val_accuracy: 0.5845 - val_loss: 1.0783
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 463ms/step - accuracy: 0.5983 - loss: 1.0657 - val_accuracy: 0.5666 - val_loss: 1.2710
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 470ms/step - accuracy: 0.6181 - loss: 1.0376 - val_accuracy: 0.6143 - val_loss: 1.0360
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 467ms/step - accuracy: 0.6312 - loss: 0.9555 - val_accu

In [8]:
ml_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(img_size, img_size),
    label_mode='int',
    batch_size=None
)

X = []
y = []
for img, label in ml_dataset:
    X.append(img.numpy())
    y.append(label.numpy())

X = np.array(X)
y = np.array(y)

# Flatten for ML models
X_flat = X.reshape(X.shape[0], -1)

Found 2527 files belonging to 6 classes.


In [9]:
X_balanced = []
y_balanced = []

unique_classes = np.unique(y)
max_count = max([np.sum(y == cls) for cls in unique_classes])

for cls in unique_classes:
    cls_indices = np.where(y == cls)[0]
    X_cls = X_flat[cls_indices]
    y_cls = y[cls_indices]
    
    X_resampled, y_resampled = resample(
        X_cls, y_cls,
        replace=True,
        n_samples=max_count,
        random_state=42
    )
    
    X_balanced.append(X_resampled)
    y_balanced.append(y_resampled)

X_bal = np.vstack(X_balanced)
y_bal = np.hstack(y_balanced)

print("Balanced class distribution:", np.bincount(y_bal))

Balanced class distribution: [594 594 594 594 594 594]


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_bal, y_bal, test_size=0.2, random_state=42)

# SVM
svm_model = SVC()
svm_model.fit(X_train, y_train)
print("SVM Classification Report:\n", classification_report(y_test, svm_model.predict(X_test)))

# KNN
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, y_train)
print("KNN Classification Report:\n", classification_report(y_test, knn_model.predict(X_test)))


SVM Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.87      0.92       144
           1       0.71      0.70      0.71       121
           2       0.78      0.81      0.79       113
           3       0.77      0.80      0.78        99
           4       0.73      0.79      0.76       108
           5       0.91      0.91      0.91       128

    accuracy                           0.82       713
   macro avg       0.81      0.81      0.81       713
weighted avg       0.82      0.82      0.82       713

KNN Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.82      0.75       144
           1       0.56      0.62      0.59       121
           2       0.87      0.58      0.69       113
           3       0.72      0.46      0.56        99
           4       0.54      0.69      0.61       108
           5       0.83      0.85      0.84       128

    accuracy         